In [ ]:
!gdown https://drive.google.com/uc?id=1UXIPvETrrAOv5-4wooxofXTgjO5-geeF

#!gdown https://drive.google.com/drive/folders/1yTQHIhmlL3QSXX6VgAXBC85ndDLOEayT?usp

#!gdown https://drive.google.com/uc?id=1yTQHIhmlL3QSXX6VgAXBC85ndDLOEayT

#!gdown https://drive.google.com/uc?id=1yTQHIhmlL3QSXX6VgAXBC85ndDLOEayT
#https://drive.google.com/drive/folders/1yTQHIhmlL3QSXX6VgAXBC85ndDLOEayT?usp=sharing
#https://drive.google.com/drive/folders/1CtD7m0FD0BNcYAADjWHj_hlTHHUYTkTz?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip T_S2.zip

In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D #down sampling 
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.utils import to_categorical 
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-3
EPOCHS = 20
BS = 30

In [ ]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images('/content/T_S2/Train'))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
	label = imagePath.split(os.path.sep)[-2]
	
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	data.append(image)
	labels.append(label)
# convert the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 1]
data = np.array(data) / 255.0
labels = np.array(labels)

In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=15,
	fill_mode="nearest")

In [ ]:
labels

In [ ]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)



In [ ]:
from sklearn.metrics import confusion_matrix
predY = model.predict(testX)

In [ ]:
predY

In [ ]:
labels = ['Makkah', 'Madena']
cm = confusion_matrix(testY.argmax(axis=1),predY.argmax(axis=1))
print(cm)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_matrix.png')
plt.show()

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on glaucoma Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("training&validation_accuracy.png")

In [ ]:
# serialize the model to disk
print("[INFO] saving ART_KSA model...")
model.save("ART_KSA.h5")

In [ ]:
import shutil, random, os, cv2, random

In [ ]:
from keras.preprocessing.image import img_to_array

In [ ]:
# load the image
import shutil, random, os, cv2, random
dirpath = "//content/T_S2/Test"
type_ = random.sample(os.listdir(dirpath), 1)
path = random.sample(os.listdir(os.path.join(dirpath,type_[0])), 1)
#print(path)

# load the image
image = cv2.imread(os.path.join(dirpath,type_[0],path[0]))
output =cv2.resize(image, (400, 400))
# pre-process the image for classification
image = cv2.resize(image, (224, 224))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

# classify the input image then find the indexes of the two class
print("[INFO] classifying image...")
proba = model.predict(image)[0]
idxs = np.argsort(proba)[::-1][:2]
from google.colab.patches import cv2_imshow
# loop over the indexes of the high confidence class labels
label = "Testing Image Type : {} ".format(type_[0])
cv2.putText(output, label, (10, (1 * 20) + 25), 
  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (102, 178, 255), 2)
  
for (i, j) in enumerate(idxs):
  label = "{}: {:.2f}%".format(lb.classes_[j], proba[j] * 100)
  cv2.putText(output, label, (10, ((i+2) * 30) + 25), 
	cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# show the probabilities for each of the individual labels
for (label, p) in zip(lb.classes_, proba):
	print("{}: {:.2f}%".format(label, p * 100))
cv2_imshow(output)


In [ ]:
accuracies = cross_val_score(estimator = model , X=trainX , y=trainY, cv=4)